# Llama Stack Client Examples
Use the llama_stack_client library to interact with a Llama Stack server

First let's install the required packages

In [1]:
%pip install -U llama-stack-client
%pip install -U llama-stack
%pip install -U agentops
%pip install -U python-dotenv
%pip install -U fastapi
%pip install opentelemetry-api
%pip install opentelemetry-sdk


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_stack_client import LlamaStackClient
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger
from llama_stack_client.types import UserMessage
from llama_stack_client.types.agent_create_params import AgentConfig
from llama_stack_client.lib.agents.agent import Agent
from dotenv import load_dotenv
import os
import agentops

load_dotenv()
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

agentops.init(AGENTOPS_API_KEY, default_tags=["llama-stack-client-example"], auto_start_session=False)

host = "0.0.0.0" # LLAMA_STACK_HOST
port = 5001 # LLAMA_STACK_PORT

full_host = f"http://{host}:{port}"

client = LlamaStackClient(
    base_url=f"{full_host}",
)

🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'


# Inference Canary + Agent Canary

In [3]:
### Inference Canary

agentops.start_session() # AgentOps start session

response = client.inference.chat_completion(
    messages=[
        UserMessage(
            content="hello world, write me a 3 word poem about the moon",
            role="user",
        ),
    ],
    model_id="meta-llama/Llama-3.2-1B-Instruct",
    stream=True
)

async for log in EventLogger().log(response):
    log.print()


### Agent Canary

import os
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig

LLAMA_STACK_PORT = 5001

# Replace with actual API keys for functionality
BRAVE_SEARCH_API_KEY = os.getenv("BRAVE_SEARCH_API_KEY") or "your-brave-search-api-key"

async def agent_test():
    client = LlamaStackClient(
        base_url=f"http://0.0.0.0:{LLAMA_STACK_PORT}",
    )

    available_shields = [shield.identifier for shield in client.shields.list()]
    if not available_shields:
        print("No available shields. Disable safety.")
    else:
        print(f"Available shields found: {available_shields}")
    available_models = [model.identifier for model in client.models.list()]
    if not available_models:
        raise ValueError("No available models")
    else:
        selected_model = available_models[0]
        print(f"Using model: {selected_model}")

    agent_config = AgentConfig(
        model=selected_model,
        instructions="You are a helpful assistant. Just say hello as a greeting.",
        sampling_params={
            "strategy": "greedy",
            "temperature": 1.0,
            "top_p": 0.9,
        },
        tools=[
            {
                "type": "brave_search",
                "engine": "brave",
                "api_key": BRAVE_SEARCH_API_KEY,
            }
        ],
        tool_choice="auto",
        tool_prompt_format="json",
        input_shields=available_shields if available_shields else [],
        output_shields=available_shields if available_shields else [],
        enable_session_persistence=False,
    )
    agent = Agent(client, agent_config)
    user_prompts = [
        "Hello",
        "Which players played in the winning team of the NBA western conference semifinals of 2014, please use tools",
    ]

    session_id = agent.create_session("test-session")

    for prompt in user_prompts:
        response = agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=session_id,
        )

        print(f"{response=}")

        for log in EventLogger().log(response):
            log.print()

await agent_test()

agentops.end_session("Success") # AgentOps end session

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=c25a8294-38d0-4b49-9ecb-8940ee264020


Assistant> Silent lunar glow
No available shields. Disable safety.
Using model: meta-llama/Llama-3.2-1B-Instruct
response=<generator object LlamaStackClientProvider.handle_response.<locals>.agent_generator at 0x1240c3990>
inference> Hello! How can I assist you today?
response=<generator object LlamaStackClientProvider.handle_response.<locals>.agent_generator at 0x1240c3840>
inference> brave_search.call(query="NBA Western Conference semifinals 2014 winners")
tool_execution> Tool:brave_search Args:{'query': 'NBA Western Conference semifinals 2014 winners'}
tool_execution> Tool:brave_search Response:{"query": "NBA Western Conference semifinals 2014 winners", "top_k": [{"title": "2014 NBA playoffs - Wikipedia", "url": "https://en.wikipedia.org/wiki/2014_NBA_playoffs", "description": "The <strong>2014</strong> <strong>NBA</strong> playoffs was the postseason tournament of the National Basketball Association&#x27;s 2013\u201314 season. The tournament concluded with the <strong>Western</stron

🖇 AgentOps: Session Stats - Duration: 7.9s | Cost: $0.00 | LLMs: 4 | Tools: 1 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=c25a8294-38d0-4b49-9ecb-8940ee264020


In [4]:
agentops.end_all_sessions()